<a href="https://colab.research.google.com/github/Krishnaugale353/Upgrad-Capstone/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

jhb

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gensim
from gensim.models import Word2Vec
from imblearn.over_sampling import SMOTE
import tensorflow
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
data=pd.read_csv('/content/drive/MyDrive/Final Capstone Project/Intensity.csv')

In [3]:
# Check the shape of the combined dataset
print("Dataset dimensions:", data.shape)


Dataset dimensions: (2039, 2)


In [4]:
# Display the first 5 rows of the combined dataset
data.head()


,content,intensity
0,Wants to know how the hell I can remember word...,happiness
1,Love is a long sweet dream & marriage is an al...,happiness
2,The world could be amazing when you are slight...,happiness
3,My secret talent is getting tired without doin...,happiness
4,"Khatarnaak Whatsapp Status Ever… Can\’t talk, ...",happiness


In [5]:
# Check for any missing values in the dataset
data.isnull().sum()


,0
content,0
intensity,0


In [6]:
# Count the number of each label in the 'intensity' column
data['intensity'].value_counts()


,count
intensity,
happiness,708
angriness,696
sadness,635


In [7]:
# Load NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
data.columns

Index(['content', 'intensity'], dtype='object')

In [9]:
!pip install emoji

In [10]:
import emoji

# Define a function to convert emojis to text descriptions
def convert_emojis_to_text(text):
    return emoji.demojize(text, delimiters=(" ", " "))  # Converts emojis to text descriptions

# Apply the function to your 'cleaned_content' column
data['content'] = data['content'].apply(convert_emojis_to_text)

# Example: 😀 gets converted to ':grinning_face:'
print(data['content'].head())


0    Wants to know how the hell I can remember word...
1    Love is a long sweet dream & marriage is an al...
2    The world could be amazing when you are slight...
3    My secret talent is getting tired without doin...
4    Khatarnaak Whatsapp Status Ever… Can\’t talk, ...
Name: content, dtype: object


In [11]:


# Create a lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to clean the text while retaining emojis
def clean_text(text):
    # Lowercase the text
    text = text.lower()
    # Retain emojis, remove special characters and numbers
    text = re.sub(r'[^a-z\s\U0001F600-\U0001F64F]', '', text)  # Retains emojis in the range
    # Tokenization
    words = nltk.word_tokenize(text)
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # Lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]
    # Join the words back into a single string
    return ' '.join(words)

# Apply the cleaning function to the 'content' column
data['cleaned_content'] = data['content'].apply(clean_text)
data.drop(columns=['content'], inplace=True)

# Preview the cleaned data
data.head()

,intensity,cleaned_content
0,happiness,want know hell remember word song year ago can...
1,happiness,love long sweet dream marriage alarm clock
2,happiness,world could amazing slightly strange
3,happiness,secret talent getting tired without anything g...
4,happiness,khatarnaak whatsapp status ever cant talk wife...


In [12]:
duplicate_count = data.duplicated().sum()
print("Number of duplicate entries:", duplicate_count)

Number of duplicate entries: 543


In [13]:
if duplicate_count > 0:
    duplicates = data[data.duplicated()]
    print("Duplicate entries:")
    print(duplicates)

Duplicate entries:
      intensity                                    cleaned_content
150   happiness  life give hundred reason cry show life thousan...
157   happiness  next time think wonderful beautiful thing life...
243   happiness                           birthday quote boyfriend
421   happiness                        love sweet new sweeter true
427   happiness                        nice someone care much care
...         ...                                                ...
2002    sadness  hurt still love true love isnt found built sil...
2003    sadness  hurt still love true love isnt found built sil...
2004    sadness  hurt still love true love isnt found built sil...
2005    sadness  hurt still love true love isnt found built sil...
2024    sadness  dont waste tear cry someone doesnt even deserv...

[543 rows x 2 columns]


In [14]:
# Remove duplicate entries
data = data.drop_duplicates()

# Check the new dimensions of the dataset
print("New dataset dimensions:", data.shape)


New dataset dimensions: (1496, 2)


In [15]:
# Count the number of each label in the 'intensity' column
data['intensity'].value_counts()


,count
intensity,
happiness,698
angriness,413
sadness,385


In [16]:
import numpy as np
from gensim.models import Word2Vec

# Step 1: Tokenize the cleaned content
sentences = [sentence.split() for sentence in data['cleaned_content']]  # Tokenize each sentence into words

# Step 2: Train a Word2Vec model using CBOW (sg=0) or Skip-gram (sg=1)
word2vec_model = Word2Vec(
    sentences,
    vector_size=300,  # Dimension of the word vectors
    window=5,         # Context window size
    sg=1,             # Use Skip-gram model
    min_count=1,      # Minimum word frequency threshold
    workers=4         # Number of parallel threads
)

# Save the model if needed
word2vec_model.save("word2vec_model.model")

# Step 3: Create a function to get the average embedding for each sentence
def get_sentence_embedding(sentence, model, embedding_dim=300):
    words = sentence.split()
    embedding_matrix = [model.wv[word] for word in words if word in model.wv]

    if embedding_matrix:
        return np.mean(embedding_matrix, axis=0)
    else:
        return np.zeros(embedding_dim)

# Step 4: Generate sentence embeddings using the trained model
X_embeddings = np.array([get_sentence_embedding(sentence, word2vec_model, 300) for sentence in data['cleaned_content']])

# Prepare the target variable
y = data['intensity']


NameError: name 'y_train' is not defined

In [18]:
# Apply SMOTE to balance the classes after creating embeddings
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_embeddings, y)

# Check the new class distribution
print("Class distribution after SMOTE:")
print(pd.Series(y_resampled).value_counts())

Class distribution after SMOTE:
intensity
happiness    698
angriness    698
sadness      698
Name: count, dtype: int64


In [19]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


In [20]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform the labels to integer values
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)  # Make sure to transform the test set only


In [21]:
X_train_reshaped = X_train.reshape(X_train.shape[0], 1, X_train.shape[1]).astype(np.float32)
X_test_reshaped = X_test.reshape(X_test.shape[0], 1, X_test.shape[1]).astype(np.float32)

In [22]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dropout(0.2))  # Add dropout to prevent overfitting
model.add(LSTM(32))  # Second LSTM layer
model.add(Dropout(0.2))  # Dropout again
model.add(Dense(3, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [23]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 1, 64)               │          93,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 3)                   │              99 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 105,955 (413.89 KB)

 Trainable params: 105,955 (413.89 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [25]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [26]:
history = model.fit(X_train_reshaped, y_train_encoded,
                    validation_split=0.2,
                    epochs=20,
                    batch_size=32,
                    callbacks=[early_stopping])


Epoch 1/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.3298 - loss: 1.0982 - val_accuracy: 0.3791 - val_loss: 1.0950
Epoch 2/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3332 - loss: 1.0977 - val_accuracy: 0.3881 - val_loss: 1.0943
Epoch 3/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3562 - loss: 1.0982 - val_accuracy: 0.3970 - val_loss: 1.0929
Epoch 4/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3489 - loss: 1.0960 - val_accuracy: 0.4090 - val_loss: 1.0929
Epoch 5/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3790 - loss: 1.0959 - val_accuracy: 0.4090 - val_loss: 1.0919
Epoch 6/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3753 - loss: 1.0933 - val_accuracy: 0.4000 - val_loss: 1.0920
Epoch 7/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3511 - loss: 1.0939 - val_accuracy: 0.3015 - val_loss: 1.1008
Epoch 8/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3649 - loss: 1.0938 - val_accuracy: 0.4060 - val_loss

In [27]:
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test_encoded)
print(f'Test Accuracy: {test_accuracy:.4f}')

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3829 - loss: 1.0896
Test Accuracy: 0.3866
